In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import cvlib
import statistics as st
import argparse
from MLDatasetBuilder import *
import os
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay 
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
import joblib

from sklearn.model_selection import GridSearchCV

In [ ]:
def view(img, title=None):
    k = 5
    fig,ax1 = plt.subplots(figsize=(k,k))
    if len(img.shape)==2:
        ax1.imshow(img, vmin=0, vmax=255, cmap='gray')
    else:
        ax1.imshow(img)
    if title:
        plt.title(title)   
    plt.axis('off')
    plt.show()

In [ ]:
def imgpad(image, r):
    """_summary_
        La función empieza obteniendo el valor de columnas y filas de la imagen original. 
        Eso nos sirve, porque el padding es agregar un "marco" a la imagen, por lo que sabemos que
        r * 2 nos generara los espacios necesarios para el nuevo lenght de cada fila de la imagen. 
                                                                                    filas       columnas
        Por lo tanto, se crea un array con np.zero de numpy y sus dimensiones son: (row+(r*2), column+(r*2)
        Por ejemplo, si la imagen es de 8 * 8, tendremos una de 10 * 10, para tener 1 pixel de marco con el padding. 
        
        Para poner la imagen dentro de la matriz, le caemos encima a el espacio que equivale a donde estaría la imagen
        original. Eso nos trae justamente el espacio en donde corresponde nuestra imagen inicial. 
    Args:
        image (uint8): array de la imagen
        r (int): cantidad de pixeles que queremos nuestro padding. 
    """
    row,column = image.shape[0:2]
    arr = np.zeros((row+(r*2), column+(r*2)))
    #arr = np.full((row+(r*2), column+(r*2)), 255)
    #print(arr)
    arr[r:r+row, r:r+column] = image
    return arr

In [28]:
folder = './fotos_placas/CNN letter Dataset/'
sub_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]
fotos = []
labels = []

for i in sub_folders:
    dir_path = folder + "/" + i

    for file_path in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, file_path)):
            image_path = dir_path + "/" + file_path
            image = cv.imread(image_path,cv.IMREAD_GRAYSCALE)
            image = cv.adaptiveThreshold(image, 255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV,51,5)
            image = imgpad(image, 3)
            #print(image.shape)
            image_2d = image.flatten()
            #print(image_2d.shape)
            fotos.append(image_2d)
            labels.append(i)
        #     view(image)
        # break

In [ ]:
print(len(fotos))

In [ ]:
from sklearn.model_selection import train_test_split
fotos_train, fotos_test, labels_train, labels_test = train_test_split(fotos, labels, test_size=0.33, random_state=100, stratify=labels)

In [ ]:
print(labels_train[54])

In [ ]:
model = LogisticRegression()
model.fit(fotos_train, labels_train)

In [ ]:
y_pred = model.predict(fotos_test)
accuracy = accuracy_score(labels_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
joblib.dump(model, "modelo_fotos.pkl")